# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** A binary classification problem. Because the output label are discrete (even binary in this case), that is "yes/no" (Yes stands for interventions are needed, no stands for interventions are not needed. ). Regression is used when we need continuous output. 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [50]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
import os

if os.getcwd().split('/')[-1] != 'student_intervention':
    os.chdir('projects/student_intervention')

student_data = pd.read_csv("student-data.csv")
print("Student data read successfully!")

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [51]:
print(student_data.columns)

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'passed'],
      dtype='object')


In [52]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed']=='yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed'] == 'no'])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/n_students

# Print the results
print("Total number of students: {}".format(n_students))
print("Number of features: {}".format(n_features))
print("Number of students who passed: {}".format(n_passed))
print("Number of students who failed: {}".format(n_failed))
print("Graduation rate of the class: {:.2f}%".format(100. * grad_rate))

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [53]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns

print("Feature columns:\n{}".format(feature_cols))
print("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X = student_data[feature_cols]
y = student_data[target_col]

# Show the feature information by printing the first five rows
print("\nFeature values:")
print(X.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [54]:
from sklearn.preprocessing import minmax_scale
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
            
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix=col)
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_train = preprocess_features(X)
print("Processed feature columns ({} total features):\n{}".format(len(X_train.columns), list(X_train.columns)))
# y_all = preprocess_features(y_all)

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


In [55]:
X_train = minmax_scale(X_train)

In [56]:
print(len(X.columns), '\n', X.columns)
print('Original shape: ', X.shape, ' after shape is: ', X_train.shape)

30 
 Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences'],
      dtype='object')
Original shape:  (395, 30)  after shape is:  (395, 48)


In [57]:
y_train = y.replace(['no', 'yes'], [0, 1])

### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [59]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=95, random_state=1, stratify=y_train)

In [60]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(300, 48) (95, 48) (300,) (95,)


In [64]:
# Save the model training and testing data to a h5 file, since test dataset has to be the same for each training 
# Or comparison of pipeline may not be accurate
import h5py 
h5_f = h5py.File('./data.h5')
dset_X_train = h5_f.create_dataset('X_train', data=X_train)
dset_X_test = h5_f.create_dataset('X_test', data=X_test)
dset_y_train = h5_f.create_dataset('y_train', data=y_train)
dset_y_test = h5_f.create_dataset('y_test', data=y_test)
h5_f.close()

#### Reload preprocessed data to keep training, test data consistent

In [65]:
import h5py 
h5_f = h5py.File('./data.h5')
X_train = h5_f['X_train'][:]
y_train = h5_f['y_train'][:]
X_test = h5_f['X_test'][:]
y_test = h5_f['y_test'][:]

In [66]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(300, 48) (95, 48) (300,) (95,)


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the **data and each model's strengths and weaknesses**. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

<table>
  <tbody>
    <tr>
      <th>Tables</th>
      <th align="left">Ensemble Methods(Gradient boosting)</th>
      <th align="left">SVM</th>
      <th align = "left">Logistic regression</th>
    </tr>
    <tr>
      <td>Real-world application</td>
      <td align="left">
          <ul>
              <li>Lots of Kaggle competition winning algorithm.</li>
              <li>Tencent data platform use distributed XGBoost for clicking through prediction in wechat shopping and lookalikes. The problems involve hundreds millions of users and thousands of features.
              <a href="https://github.com/dmlc/xgboost/tree/master/demo#usecases">link</a>
              </li>
          </ul>
      </td>
      <td align="left">A company named Directing in Greece uses SVM for customers annual churn prediction [link](http://www.kdnuggets.com/2014/09/automotive-customer-churn-prediction-svm-som.html) </td>
      
      <td align = "left">Logistic regression analysis are used in banks to identify demographic and behavioral characteristics associated with likelihood to default on a bank loan [link](http://smartdrill.com/logistic-regression.html)</td>
    </tr>
   <tr>
      <td>Strengths of the model</td>
      <td align="left">
          <ul>
          <li>Easy to interpret (Tree based model) and invariant to input scale[Link](https://www.quora.com/Why-is-xgboost-given-so-much-less-attention-than-deep-learning-despite-its-ubiquity-in-winning-Kaggle-solutions/answer/Tianqi-Chen-1?srid=32HW) </li>
          <li>High Accuracy (Kaggle 1st place winner Owen Zhang: When in doubt, use xgboost (for tabular data only I think) )</li>
          <li>No difficult parameters/architectures to set</li>
          <li>Computationally efficient, comparing to neural nets</li>
          <li>All boosting, including gradient boost machine, are additive models, which means later model relys on its previous model. Therefore, it is hard to do parallel training, while it is possible recently [link](http://dmlc.ml/2016/12/14/GPU-accelerated-xgboost.html)</li>
          <li>Regularization is important to xgboost as Tianqi Chen added it in its objective function. </li>
          <li>Naturally handling heterogeneous features</li>
          <li>Robustness to outliers in output space</li>
          <li>Automatically detects feature interactions [link](http://www.slideshare.net/DataRobot/gradient-boosted-regression-trees-in-scikitlearn)</li>
          <li>Support different loss functions </li>
          </ul> 
      </td>
      <td align="left">
          <ul>
              <li>Versatile kernels can be used </li>
              <li>Good at dealing with training data in high dimensional space</li>
          </ul>
      </td>
      <td align = "left"><ul>
      <li>Fast to train</li>
      <li>Simple, easy to interpret, result can be used as baseline</li>
      <li>Since its output is probability, it can be used for ranking</li>
      <li>Robust to noise</li>
      </ul></td>
    </tr>
    <tr>
      <td>Weakness of the model</td>
      <td align="left">
          <ul>
              <li>Feature engineer intense (lots of winning kaggler mentioned how important feature enginnering is comparing to neural nets)</li>
              <li>require careful tuning</li>
              <li>Slow to train (But fast to predict)</li>
              <li>Not efficient to skewed distribution, since underline it is a tree model ensembler, weights to unbalanced samples can be set to solve the problem. </li>
              <li>Needs more data comparing to SVM and logistic regression</li>
              <li>Take more time and resources comparing to SVM and logistic regression</li>
          </ul>
      </td>
      <td align="left">
      <ul>
          <li>Inefficient to train if lots of training examples [link](https://www.quora.com/What-are-the-advantages-of-different-classification-algorithms/answer/Xavier-Amatriain?srid=32HW)</li>
          <li>Hard to attribute meanings in high dimension comparing to decision trees</li>
          <li>Slow when training large dataset</li>
      </ul>
      </td>
      <td align = "left">
          <ul>
              <li>Relatively low accuracy</li>
              <li>Linear Combination of features</li>
              <li>Hardly to handle categorical features</li>
          </ul>
      </td>
    </tr>
  </tbody>
</table>

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [67]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target, y_pred, pos_label=1)


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))
    print()

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [68]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import numpy as np
# from skearln import model_C

# TODO: Initialize the three models
clf_A = GradientBoostingClassifier()
clf_B = SVC()
clf_C = LogisticRegression()
clf_list = [clf_A, clf_B, clf_C]

# TODO: Set up the training set sizes
X_train_100_index = np.random.choice(300, size=100, replace=False)

X_train_100 = X_train[X_train_100_index]
y_train_100 = y_train[X_train_100_index]

X_train_200_index = np.random.choice(300, size=200, replace=False)
X_train_200 = X_train[X_train_200_index]
y_train_200 = y_train[X_train_200_index]

X_train_300 = X_train
y_train_300 = y_train

X_train_list = [X_train_100, X_train_200, X_train_300]
y_train_list = [y_train_100, y_train_200, y_train_300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
for X, y in zip(X_train_list, y_train_list):
    for clf in clf_list:
        train_predict(clf, X, y, X_test, y_test)

Training a GradientBoostingClassifier using a training set size of 100. . .
Trained model in 0.1010 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.8219.

Training a SVC using a training set size of 100. . .
Trained model in 0.0014 seconds
Made predictions in 0.0008 seconds.
F1 score for training set: 0.8372.
Made predictions in 0.0007 seconds.
F1 score for test set: 0.8050.

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0008 seconds
Made predictions in 0.0019 seconds.
F1 score for training set: 0.8917.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7755.

Training a GradientBoostingClassifier using a training set size of 200. . .
Trained model in 0.0820 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 0.9964.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.7826.

Training a SVC using a training set size of

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Gradient Boost Machine**  

| Training Set Size | Training Time(sec) | Prediction Time (test sec) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |           0.1010              |       0.0006                 |   1.0000               |      0.8219 (Every time, it is different and its difference is huge (over 10% sometimes, unstable))          |
| 200               |       0.0820          |          0.0004             |       0.9964           |       0.7826          |
| 300               |       0.1070                  |      0.0004                  |       0.9781           |    0.7626 (Not stable, but better comparing to 100 training samples)      |

** Classifer 2 - SVM**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0014                   |         0.0007              |  0.8372                |         0.8050        |
| 200               |       0.0053           |         0.0021               |        0.8131          |     0.8050            |
| 300               |     0.0081                    |        0.0019                |             0.8024     |     0.8050(Not changed, comparing to previous run)     |

** Classifer 3 - Logistic regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0008                 |       0.0002                 |          0.8917        |     0.7755            |
| 200               |       0.0034                  |        0.0006                |          0.8629        |    0.7591             |
| 300               |         0.0016                  |        0.0002                 |         0.8326         |      0.7591 (Not changed, compare to previous run)          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

- From my perspective, SVM is the best fit for this problem based on experiments earlier, for its performance on test set is the best, its training and prediction time is fast and the dataset is relatively small.  

- It's very tempting to used Gradient boost machine since it gets such a high reputation in Kaggle competitions for tabular dataset, and its training set F1 score is significantly higher. It is possible to get higher accuracy by fine tuning hyper-parameters. But according to Occam's Razor principle: Use the least complicated algorithm that can address the needs and only go for something more complicated if strictly necessary. Therefore, I think SVM is the best model in this model. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

- Support vector machine(SVM) is a classical while effective algorithm in machine learning. It fits for solving one-to-one classification problem like the this problem. 
- In this problem, it considers two classes (Graduated/Dropout) as 2 clusters, therefore, it tried to find a hyperplane(bountry) which can maximize margins distance between the 2 clusters. In other words, it is trying its best to draw a boudary between two clusters to make them as seperable as possible. Therefore, if a new point comes in, it can locate it in the space and make prediction of its class. As show in this diagram 
![Image of svm](./img/svm.png)

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [69]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from time import time
from sklearn.svm import SVC
import numpy as np
from sklearn.metrics import f1_score

# TODO: Create the parameters list you wish to tune
parameters = {
    'C': np.linspace(0.5, 3, 50),
    'gamma': np.linspace(0.001, 0.3, 50)
}

# TODO: Initialize the classifier
clf = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer, n_jobs=-1)

start_time = time()
# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)
print('Time used is: ', time() - start_time)
# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
f1_train = predict_labels(clf, X_train, y_train)
f1_test = predict_labels(clf, X_test, y_test)
print("Tuned model has a training F1 score of {:.4f}.".format(f1_train))
print("Tuned model has a testing F1 score of {:.4f}.".format(f1_test))

Time used is:  35.52073574066162
Made predictions in 0.0061 seconds.
Made predictions in 0.0020 seconds.
Tuned model has a training F1 score of 0.8247.
Tuned model has a testing F1 score of 0.8105.


In [70]:
print(clf.C)
print(clf.gamma)

1.0612244898
0.0559183673469


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

- Final F<sub>1</sub> score for training is 0.8247 and for testing is 0.8105.
- Comparing to the untuned model, its training and testing score both increased, but not much (2.23% increased for training F1 score and 0.55% increased comparing to untuned model with 300 training data )
- I think the reason is that
    - the dataset is small, both in example size and dimensional size. Therefore, there are not much extra information squeezed by tuning hyper-parameters.
    - I just used min_max_scale to preprocess the features, further careful feature engineering can help to improve the predictive ability. 

#### Also, try fine tune Gradient boost machine and random forest

In [39]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from time import time
from sklearn.ensemble import GradientBoostingClassifier

# TODO: Create the parameters list you wish to tune
parameters = {
    'learning_rate': [0.01, 0.005, 0.002],
    'max_depth': [2, 3, 4, 6],
    'min_samples_leaf': [3, 5, 9, 17],
    'max_features': [1.0, 0.3, 0.1], 
    'n_estimators': [100, 200, 300, 400, 500]
}

clf = GradientBoostingClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer, n_jobs=-1)

start_time = time()
# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)
print('Time used is: ', time() - start_time)
# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
f1_train = predict_labels(clf, X_train, y_train)
f1_test = predict_labels(clf, X_test, y_test)


print("Tuned model has a training F1 score of {:.4f}.".format(f1_train))
print("Tuned model has a testing F1 score of {:.4f}.".format(f1_test))



Time used is:  217.45741844177246
Made predictions in 0.0032 seconds.
Made predictions in 0.0011 seconds.
Tuned model has a training F1 score of 0.8499.
Tuned model has a testing F1 score of 0.8129.


In [40]:
print(clf.learning_rate, clf.max_depth, clf.min_samples_leaf, clf.max_features, clf.n_estimators)

0.005 6 5 0.1 200


In [9]:
from sklearn.ensemble import RandomForestClassifier
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from time import time
from sklearn.ensemble import GradientBoostingClassifier

# TODO: Create the parameters list you wish to tune
parameters = {
    'max_depth': [2, 3, 4, 6],
    'min_samples_leaf': [1, 3, 5, 9, 17],
    'max_features': [1.0, 0.3, 0.1], 
    'n_estimators': [10, 50, 100, 200, 300, 400, 500]
}

clf = RandomForestClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer, n_jobs=-1)

start_time = time()
# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)
print('Time used is: ', time() - start_time)
# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
f1_train = predict_labels(clf, X_train, y_train)
f1_test = predict_labels(clf, X_test, y_test)


print("Tuned model has a training F1 score of {:.4f}.".format(f1_train))
print("Tuned model has a testing F1 score of {:.4f}.".format(f1_test))



Time used is:  211.70204544067383
Made predictions in 0.0020 seconds.
Made predictions in 0.0012 seconds.
Tuned model has a training F1 score of 0.8170.
Tuned model has a testing F1 score of 0.8143.


In [11]:
print(clf.max_depth, clf.min_samples_leaf, clf.max_features, clf.n_estimators)

2 17 1.0 10


**Answer: **

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.